In [15]:
import cv2
import os
import numpy as np


class Drawing:
    def __init__(self, class_names):
        self.class_names = class_names
        self.class_colors = {
            0: (0, 255, 0),  # Green
            1: (0, 165, 255),  # Orange
            2: (0, 0, 255),  # Red
            3: (255, 0, 0),  # Blue
            4: (255, 255, 0)  # Cyan
        }

    def draw_bounding_box(self, image, class_id, x_center, y_center, width, height):
        h, w = image.shape[:2]

        # Convert normalized coordinates to pixel coordinates
        x_center_px = int(x_center * w)
        y_center_px = int(y_center * h)
        width_px = int(width * w)
        height_px = int(height * h)

        # Calculate top-left corner
        x1 = int(x_center_px - width_px / 2)
        y1 = int(y_center_px - height_px / 2)
        x2 = int(x_center_px + width_px / 2)
        y2 = int(y_center_px + height_px / 2)

        # Get color for this class
        color = self.class_colors.get(class_id, (255, 255, 255))

        # Draw rectangle
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)

        # Draw class label
        class_name = self.class_names[class_id]
        cv2.putText(image, class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        return image

    def draw_polygon(self, image, class_id, coords):
        h, w = image.shape[:2]

        # Convert normalized coordinates to pixel coordinates
        points = []

        # noinspection PyTypeChecker
        for i in range(0, len(coords), 2):
            x_px = int(coords[i] * w)
            y_px = int(coords[i + 1] * h)
            points.append([x_px, y_px])

        # Convert to numpy array
        points = np.array(points, dtype=np.int32)

        # Get color for this class
        color = self.class_colors.get(class_id, (255, 255, 255))

        # Draw filled polygon with transparency
        overlay = image.copy()
        cv2.fillPoly(overlay, [points], color)
        image = cv2.addWeighted(image, 0.7, overlay, 0.3, 0)

        # Draw polygon outline
        cv2.polylines(image, [points], True, color, 2)

        # Draw class label at centroid
        M = cv2.moments(points)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            class_name = self.class_names[class_id]
            cv2.putText(image, class_name, (cx - 20, cy), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        return image


In [3]:
import os
import pandas as pd

images_path = "/Users/hinsun/Workspace/Software/Agrismart/datasets/rice-leaf-diseases/train/images"

labels_path = "/Users/hinsun/Workspace/Software/Agrismart/datasets/rice-leaf-diseases/train/labels"

image_files = os.listdir(images_path)
label_files = os.listdir(labels_path)

count_file_health = 0
for image_file in image_files:
    label_file = image_file.replace(".jpg", ".txt")
    if label_file not in label_files:
        print(f"Label file for {image_file} is missing.")
        continue

    with open(os.path.join(labels_path, label_file), "r") as f:
        labels = f.readlines()

    if len(labels) == 0:
        continue

    class_id = None
    for label in labels:
        parts = label.strip().split()
        class_id = int(parts[0])

    if class_id == 2:
        count_file_health += 1

print(f"Healthy image found: {count_file_health}")

Image: brown_val-77-_jpg.rf.82c5100787ea7aeb8802940ddc3c4b8e.jpg, Class ID: 1
Image: brown_val-77-_jpg.rf.82c5100787ea7aeb8802940ddc3c4b8e.jpg, Class ID: 1
Image: brown_val-77-_jpg.rf.82c5100787ea7aeb8802940ddc3c4b8e.jpg, Class ID: 1
Image: brown_val-77-_jpg.rf.82c5100787ea7aeb8802940ddc3c4b8e.jpg, Class ID: 1
Image: brown_val-77-_jpg.rf.82c5100787ea7aeb8802940ddc3c4b8e.jpg, Class ID: 1
Image: narrow_brown-47-_jpg.rf.a1b4bb902793f04968d87aeaf4935bf4.jpg, Class ID: 7
Image: narrow_brown-47-_jpg.rf.a1b4bb902793f04968d87aeaf4935bf4.jpg, Class ID: 7
Image: narrow_brown-47-_jpg.rf.a1b4bb902793f04968d87aeaf4935bf4.jpg, Class ID: 7
Image: narrow_brown-47-_jpg.rf.a1b4bb902793f04968d87aeaf4935bf4.jpg, Class ID: 7
Image: img_brown_12_jpg.rf.8918fb21bdb61e5bd05af0dbe3c06607.jpg, Class ID: 1
Image: img_brown_12_jpg.rf.8918fb21bdb61e5bd05af0dbe3c06607.jpg, Class ID: 1
Image: img_brown_12_jpg.rf.8918fb21bdb61e5bd05af0dbe3c06607.jpg, Class ID: 1
Image: img_brown_12_jpg.rf.8918fb21bdb61e5bd05af0dbe3c0

In [18]:
classnames = [
    "Bacterial Leaf Blight",
    "Brown Spot",
    "Healthy",
    "Leaf Blast",
    "Leaf Blight",
    "Leaf Scald",
    "Leaf Smut",
    "Narrow Brown Spot",
]


def show_single_sample(image_file):
    label_file = image_file.replace(".jpg", ".txt")

    image_path = os.path.join(images_path, image_file)
    label_path = os.path.join(labels_path, label_file)

    image = cv2.imread(image_path)

    with open(label_path, "r") as f:
        labels = f.readlines()

    drawing = Drawing(classnames)
    for label in labels:
        parts = label.strip().split()

        if len(parts) < 5:
            continue
        elif len(parts) == 5:
            # Bounding box format
            class_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            image = drawing.draw_bounding_box(
                image, class_id, x_center, y_center, width, height
            )
        else:
            class_id = int(parts[0])
            coordinates = [float(coord) for coord in parts[1:]]

            image = drawing.draw_polygon(image, class_id, coordinates)

    cv2.imshow(f"Annotated: {image_file}", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


show_single_sample("brown_spot-102-_jpg.rf.2393a056cc5b78430c2d1fc7451a074f.jpg")